In [10]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

In [33]:
df =  pd.read_csv('data.csv')
df.head()
df.dropna(inplace= True)

In [34]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [35]:
X.head()

,T,TM,Tm,SLP,H,VV,V,VM
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2


In [36]:
y.head()

0    219.720833
1    182.187500
2    154.037500
3    223.208333
4    200.645833
Name: PM 2.5, dtype: float64

In [37]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers',2,20)):
        model.add(layers.Dense(units = hp.Int('units_' + str(i),
                                              min_value = 32,
                                              max_value = 512,
                                              step = 32),
                               activation = 'relu'))
    model.add(layers.Dense(1, activation = 'linear'))
    model.compile(
        optimizer = keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2,1e-3,1e-4])),
        loss = 'mean_absolute_error',
        metrics = ['mean_absolute_error'])
    return model

In [38]:
tuner = RandomSearch(
    build_model,
    objective = 'val_loss',
    max_trials = 5
)

INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [39]:
tuner.search_space_summary()

Search space summary
Default search space size: 19
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_6

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3,random_state = 42)


In [41]:
tuner.search(X_train, y_train, epochs = 10,validation_data = (X_test, y_test))


Trial 5 Complete [00h 00m 02s]
val_loss: 59.198997497558594

Best val_loss So Far: 59.198997497558594
Total elapsed time: 00h 00m 04s
INFO:tensorflow:Oracle triggered exit


In [42]:
best_model = tuner.get_best_models()[0]            

In [51]:
best_hps=tuner.get_best_hyperparameters()[0]

In [52]:
model = tuner.hypermodel.build(best_hps)

In [53]:
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
20/20 [==============================] - 1s 6ms/step - loss: 73.8834 - mean_absolute_error: 73.8834 - val_loss: 69.5757 - val_mean_absolute_error: 69.5757
Epoch 2/50
20/20 [==============================] - 0s 3ms/step - loss: 67.9526 - mean_absolute_error: 67.9526 - val_loss: 70.4502 - val_mean_absolute_error: 70.4502
Epoch 3/50
20/20 [==============================] - 0s 3ms/step - loss: 67.3923 - mean_absolute_error: 67.3923 - val_loss: 69.4932 - val_mean_absolute_error: 69.4932
Epoch 4/50
20/20 [==============================] - 0s 3ms/step - loss: 67.3235 - mean_absolute_error: 67.3235 - val_loss: 69.6750 - val_mean_absolute_error: 69.6750
Epoch 5/50
20/20 [==============================] - 0s 3ms/step - loss: 67.1963 - mean_absolute_error: 67.1963 - val_loss: 69.1593 - val_mean_absolute_error: 69.1593
Epoch 6/50
20/20 [==============================] - 0s 3ms/step - loss: 67.4386 - mean_absolute_error: 67.4386 - val_loss: 69.3819 - val_mean_absolute_error: 69.3819
Epoc